# Problems Jupyter Notebook

## Problem 1: Extending the Lady Tasting Tea
![The Lady Tasting Tea](https://upload.wikimedia.org/wikipedia/en/2/2d/The_Lady_Tasting_Tea_-_David_Salsburg.jpg)

## Problem 2: Normal Distribution

## Problem 3: t-Tests

## Problem 4: ANOVA

### References:
- A very helpful github guide on `markdown` used throughout for all markdown formatting [here](https://github.com/adam-p/markdown-here/wiki/markdown-cheatsheet)

## END